In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Install transformers library.
!pip install -q git+https://github.com/huggingface/transformers.git
# Install helper functions.
!pip install -q git+https://github.com/gmihaila/ml_things.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 4.3 MB/s 
     |████████████████████████████████| 86 kB 4.6 MB/s 
     |████████████████████████████████| 6.6 MB 48.5 MB/s 
     |████████████████████████████████| 53 kB 1.3 MB/s 
     |████████████████████████████████| 11.2 MB 3.8 MB/s 
     |████████████████████████████████| 930 kB 54.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
! git clone https://github.com/SKT-AI/KoGPT2

Cloning into 'KoGPT2'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 216 (delta 50), reused 45 (delta 17), pack-reused 121
Receiving objects: 100% (216/216), 656.89 KiB | 1.23 MiB/s, done.
Resolving deltas: 100% (109/109), done.


In [4]:
import io
import os
import torch
import pandas as pd
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.

In [ ]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 
# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token

In [13]:
! pwd

/content


In [18]:
data_path = os.path.join("./drive/MyDrive/Colab_Notebooks/voc_rm/datasets/dktc/data/insert_koen", "train.csv")

df = pd.read_csv(data_path)

df.head()

,idx,label,class,conversation
0,0,악성민원상담,협박,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,악성민원상담,협박,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,악성민원상담,기타 괴롭힘,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,악성민원상담,갈취,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,악성민원상담,갈취,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [20]:
sent = df['conversation'][0]

len(sent)

242

In [21]:
inputs = tokenizer(sent, return_tensors="pt", padding=True, truncation=True,  max_length=400)

inputs

{'input_ids': tensor([[ 9782, 10099, 24775, 30707, 24833,  9831,  8104,  9033, 12657, 19635,
         35691, 12521, 11482,  7832, 11387, 14909,  9122,  7532, 16288, 40401,
          9351,  9168, 20683, 31424, 12906,  9098,   406, 23971, 30707, 40809,
         13358,  6866,  9585, 29205, 12222,  8718, 10010, 10099,  6824, 11814,
          8711,   389, 10099,  6824, 14909,  7383,  9873, 24173, 30707,  8239,
         21598, 11482,  7832, 37194, 29205, 12222,  8718, 10010, 10099, 12333,
         11814, 11067, 10811,   389, 11814,  9350, 16337, 10099,  8066,  9873,
         10464,  9168,  9079, 28936, 30707,  7621,  7481,  6853,  7991,  9585,
         11814,  9350, 26616, 16691, 15354,  7489, 35498, 41547, 25689,  9054,
         30707,  7621, 27436,  6872,  6921,   389, 15577, 33473, 10604,  9037,
          7601, 35498,  7801, 25856]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [27]:
tmp = inputs['input_ids']



tensor([ 9782, 10099, 24775, 30707, 24833,  9831,  8104,  9033, 12657, 19635,
        35691, 12521, 11482,  7832, 11387, 14909,  9122,  7532, 16288, 40401,
         9351,  9168, 20683, 31424, 12906,  9098,   406, 23971, 30707, 40809,
        13358,  6866,  9585, 29205, 12222,  8718, 10010, 10099,  6824, 11814,
         8711,   389, 10099,  6824, 14909,  7383,  9873, 24173, 30707,  8239,
        21598, 11482,  7832, 37194, 29205, 12222,  8718, 10010, 10099, 12333,
        11814, 11067, 10811,   389, 11814,  9350, 16337, 10099,  8066,  9873,
        10464,  9168,  9079, 28936, 30707,  7621,  7481,  6853,  7991,  9585,
        11814,  9350, 26616, 16691, 15354,  7489, 35498, 41547, 25689,  9054,
        30707,  7621, 27436,  6872,  6921,   389, 15577, 33473, 10604,  9037,
         7601, 35498,  7801, 25856])

In [28]:
len(tmp[0])

104